# 古典ランダムウォーク（DeepWalkとNode2Vec）

## 概要
この記事では、「古典ランダムウォーク（DeepWalkとNode2Vec）」の定義や性質、応用例について詳しく説明する。ランダムウォークを用いたグラフ埋め込み手法であるDeepWalkとNode2Vecの数学的背景、アルゴリズムの詳細、具体的なPythonコードによる実装例を通じて、推薦システムなどでの利用例やメリット・デメリットを解説する。

## ソースコード

### github
- jupyter notebook形式のファイルは[こちら](https://github.com/hiroshi0530/wa-src/blob/master/rec/qwalk/02/02_nb.ipynb)

### google colaboratory
- google colaboratory で実行する場合は[こちら](https://colab.research.google.com/github/hiroshi0530/wa-src/blob/master/rec/qwalk/02/02_nb.ipynb)


## 実行環境
OSはmacOSである。LinuxやUnixのコマンドとはオプションが異なることに注意。

In [1]:
!sw_vers

ProductName:		macOS
ProductVersion:		13.5.1
BuildVersion:		22G90


In [2]:
!python -V

Python 3.9.17


pandasのテーブルを見やすいようにHTMLのテーブルにCSSの設定を行う。

In [ ]:
from IPython.core.display import HTML

style = """
<style>
    .dataframe thead tr:only-child th {
        text-align: right;
    }

    .dataframe thead th {
        text-align: left;
        padding: 5px;
    }

    .dataframe tbody tr th {
        vertical-align: top;
        padding: 5px;
    }

    .dataframe tbody tr:hover {
        background-color: #ffff99;
    }

    .dataframe {
        background-color: white;
        color: black;
        font-size: 16px;
    }

</style>
"""
HTML(style)

基本的なライブラリをインポートし watermark を利用してそのバージョンを確認する。
ついでに乱数のseedの設定を行う。

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [2]:
import random

import scipy
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

seed = 123
random_state = 123

random.seed(seed)
np.random.seed(seed)


from watermark import watermark

print(watermark(python=True, watermark=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.9.17
IPython version      : 8.17.2

scipy     : 1.11.2
numpy     : 1.25.2
matplotlib: 3.8.1

Watermark: 2.4.3



In [ ]:
## 古典ランダムウォークとは

### 定義
古典ランダムウォークとは、グラフ理論におけるランダムウォークの一種である。グラフ上のノードからスタートし、ランダムに隣接ノードへと移動するプロセスを繰り返すものである。このプロセスは確率的であり、次のノードへの移動はランダムに決定される。

ランダムウォークの確率行列 $\mathbf{P}$ は次のように定義される：

$$
\mathbf{P}_{ij} = \begin{cases}
\frac{1}{d_i} & \text{if } (i, j) \in E \\
0 & \text{otherwise}
\end{cases}
$$

ここで、$d_i$ はノード $i$ の次数である。

### 性質
- **エルゴード性**：十分な長さのランダムウォークはグラフ全体をカバーする。
- **定常分布**：長い時間をかけると、ランダムウォークは定常分布に収束する。
- **マルコフ性**：現在の状態のみが次の状態に依存し、過去の状態には依存しない。

## DeepWalk

### 定義
DeepWalkは、グラフ内のノードをランダムウォークに基づいて学習する手法である。具体的には、グラフ上でランダムウォークを行い、その結果得られたノードシーケンスを用いて、Skip-gramモデルでノードの埋め込みを学習する。

### アルゴリズム
1. **ランダムウォークの生成**：各ノードからスタートし、ランダムに隣接ノードへ移動するウォークを複数生成する。
2. **Skip-gramモデルの学習**：生成されたノードシーケンスを入力として、Skip-gramモデルを用いてノードの埋め込みを学習する。

$$
J = - \sum_{w \in V} \sum_{u \in N(w)} \log P(u|w)
$$

### Pythonによる実装例
以下に、DeepWalkの基本的な実装例を示す：



In [ ]:
import numpy as np
import networkx as nx
from gensim.models import Word2Vec
from pprint import pprint


def random_walk(graph, start_node, walk_length):
    walk = [start_node]
    for _ in range(walk_length - 1):
        neighbors = list(graph.neighbors(walk[-1]))
        if neighbors:
            walk.append(np.random.choice(neighbors))
        else:
            break
    return walk


def generate_walks(graph, num_walks, walk_length):
    walks = []
    nodes = list(graph.nodes())
    for _ in range(num_walks):
        np.random.shuffle(nodes)
        for node in nodes:
            walks.append(random_walk(graph, node, walk_length))
    return walks


# グラフの作成
G = nx.karate_club_graph()

# ランダムウォークの生成
num_walks = 10
walk_length = 5
walks = generate_walks(G, num_walks, walk_length)

# Skip-gramモデルでの学習
model = Word2Vec(walks, vector_size=64, window=5, min_count=0, sg=1)

# ノードの埋め込みを表示
embeddings = {node: model.wv[node] for node in G.nodes()}
pprint(embeddings)

In [ ]:
## Node2Vec

### 定義
Node2Vecは、DeepWalkを拡張した手法であり、ランダムウォークにおいて深さ優先探索（DFS）と幅優先探索（BFS）をバランス良く組み合わせることで、ノードの埋め込みを学習する。

### アルゴリズム
1. **ランダムウォークの生成**：DFSとBFSのバランスを調整するためのパラメータ $p$ と $q$ を導入し、ランダムウォークを生成する。
2. **Skip-gramモデルの学習**：DeepWalkと同様に、生成されたノードシーケンスを入力として、Skip-gramモデルでノードの埋め込みを学習する。

$$
J = - \sum_{w \in V} \sum_{u \in N(w)} \log P(u|w)
$$

### Pythonによる実装例
以下に、Node2Vecの基本的な実装例を示す：




In [ ]:
import numpy as np
import networkx as nx
from gensim.models import Word2Vec
from pprint import pprint


class Node2Vec:
    def __init__(self, graph, p, q, num_walks, walk_length):
        self.graph = graph
        self.p = p
        self.q = q
        self.num_walks = num_walks
        self.walk_length = walk_length

    def node2vec_walk(self, start_node):
        walk = [start_node]
        while len(walk) < self.walk_length:
            cur = walk[-1]
            cur_nbrs = list(self.graph.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(np.random.choice(cur_nbrs))
                else:
                    prev = walk[-2]
                    probs = []
                    for nbr in cur_nbrs:
                        if nbr == prev:
                            probs.append(1 / self.p)
                        elif self.graph.has_edge(prev, nbr):
                            probs.append(1)
                        else:
                            probs.append(1 / self.q)
                    probs = np.array(probs)
                    probs = probs / probs.sum()
                    walk.append(np.random.choice(cur_nbrs, p=probs))
            else:
                break
        return walk

    def generate_walks(self):
        walks = []
        nodes = list(self.graph.nodes())
        for _ in range(self.num_walks):
            np.random.shuffle(nodes)
            for node in nodes:
                walks.append(self.node2vec_walk(node))
        return walks


# グラフの作成
G = nx.karate_club_graph()

# Node2Vecのパラメータ設定
p = 1
q = 1
num_walks = 10
walk_length = 5

# Node2Vecのインスタンス生成とウォークの生成
node2vec = Node2Vec(G, p, q, num_walks, walk_length)
walks = node2vec.generate_walks()

# Skip-gramモデルでの学習
model = Word2Vec(walks, vector_size=64, window=5, min_count=0, sg=1)

# ノードの埋め込みを表示
embeddings = {node: model.wv[node] for node in G.nodes()}
pprint(embeddings)



## 応用例

### 推薦システムでの利用
- **ユーザーとアイテムの関係性をモデル化**：ユーザーとアイテムをノードとして扱い、ユーザーの行動履歴に基づくランダムウォークを行うことで、ユーザーの好みを埋め込みベクトルとして学習する。
- **ソーシャルネットワーク分析**：ユーザー間の関係をグラフとして表現し、ランダムウォークを用いて関係性の強さをモデル化する。

### メリット
- **スケーラビリティ**：大規模なグラフに対しても効率的に計算が可能である。
- **フレキシビリティ**：パラメータ $p$ と $q$ の調整により、探索の深さと幅をバランス良く調整できる。

### デメリット
- **計算コスト**：非常に大規模なグラフに対しては、計算コストが増加する可能性がある。
- **ハイパーパラメータの調整**：適切なハイパーパラメータの選定が結果に大きく影響する。

## 結論
この記事では、古典ランダムウォーク、DeepWalk、Node2Vecの定義、性質、応用例について説明した。ランダムウォークを利用したグラフ埋め込み手法は、推薦システムやソーシャルネット

ワーク分析などにおいて有効であることが示された。具体的なPythonコードを通じて、実装の流れも理解できたのではないだろうか。

### 参考文献
- Perozzi, A., Al-Rfou, R., & Skiena, S. (2014). DeepWalk: Online Learning of Social Representations. In Proceedings of the 20th ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD '14).
- Grover, A., & Leskovec, J. (2016). node2vec: Scalable Feature Learning for Networks. In Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD '16).